<a href="https://colab.research.google.com/github/danielAdama/tutorials/blob/main/store_huggingFace_model_locally.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install transformers numpy boto3 colab-env --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 31.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for colab-env: filename=colab_env-0.2.0-py3-none-any.whl size=3807 sha256=3db57a3329b45bf978ae6d49c8f9f8f8d2a8bebc4b5cb0b075d8f9ef566d75f3
  Stored in directory: /root/.cache/pip/wheels/ae/36/4f/466c2cd4db5d08f317893a920c4a0f58a81459ee3bdb136d35
Successfully built colab-env
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.1
    Uninstalling transformers-4.29.1:
      Successfully uninstalled transformers-4.29.1
ERROR: pip's dependency resolver does not currently take into account all the p

In [ ]:
import torch

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
save_path = 'humarin/chatgpt_paraphraser_on_T5_base'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

In [ ]:
!mkdir save_path

In [ ]:
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('humarin/chatgpt_paraphraser_on_T5_base/tokenizer_config.json',
 'humarin/chatgpt_paraphraser_on_T5_base/special_tokens_map.json',
 'humarin/chatgpt_paraphraser_on_T5_base/tokenizer.json')

In [ ]:
!ls humarin

chatgpt_paraphraser_on_T5_base


In [ ]:
!zip -r humarin.zip humarin

  adding: humarin/ (stored 0%)
  adding: humarin/chatgpt_paraphraser_on_T5_base/ (stored 0%)
  adding: humarin/chatgpt_paraphraser_on_T5_base/generation_config.json (deflated 29%)
  adding: humarin/chatgpt_paraphraser_on_T5_base/special_tokens_map.json (deflated 86%)
  adding: humarin/chatgpt_paraphraser_on_T5_base/config.json (deflated 64%)
  adding: humarin/chatgpt_paraphraser_on_T5_base/pytorch_model.bin (deflated 7%)
  adding: humarin/chatgpt_paraphraser_on_T5_base/tokenizer_config.json (deflated 83%)
  adding: humarin/chatgpt_paraphraser_on_T5_base/tokenizer.json (deflated 74%)


In [ ]:
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

In [ ]:
text = ["May i know your house address?", "What is your home address?"]
for i in text:
  print(paraphrase(i))

['Where is your house located?', 'What is the address of your residence?', 'Can you tell me the location of your residence?', 'Where is your house located? Request', 'Where is your house located? Requests only.']
['Where do you live?', 'What is your current place of residence?', 'Can you tell me where your place is?', "Where do you live? What's your address?", 'Do you happen to be at home?']


In [ ]:
import colab_env
colab_env.__version__
# !more gdrive/My\ Drive/vars.env

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


'0.2.0'

In [ ]:
import boto3
import os
import botocore

In [ ]:
 s3 = boto3.client(
    "s3",
    aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
    )
 
key = "humarin.zip"
bucket_name = "paraphrase-models-store"

try:
  s3.upload_file('humarin.zip', bucket_name, key)
  print('Zip file uploaded successfully.')
except botocore.exceptions.ClientError as ex:
  print('Error occurred:', ex.response['Error']['Message'])

In [ ]:
obj = boto3.client("s3") 
obj.download_file( Filename="Downloads/humarin.zip", Bucket=bucket_name, Key=key)

In [ ]:
import io
import zipfile

# Load the zip file from S3
object = s3.get_object(Bucket=bucket_name, Key=key)
buffer = io.BytesIO(object['Body'].read())
zipfile = zipfile.ZipFile(buffer)

# Process the files in the zip file
for filename in zipfile.namelist():
    # Do something with each file in the zip file
    with zipfile.open(filename) as file:
        file_contents = file.read()
        print(f'Contents of {filename}: {file_contents}')


Contents of humarin/: b''
Contents of humarin/chatgpt_paraphraser_on_T5_base/: b''
Contents of humarin/chatgpt_paraphraser_on_T5_base/generation_config.json: b'{\n  "_from_model_config": true,\n  "decoder_start_token_id": 0,\n  "eos_token_id": 1,\n  "pad_token_id": 0,\n  "transformers_version": "4.29.1"\n}\n'
Contents of humarin/chatgpt_paraphraser_on_T5_base/special_tokens_map.json: b'{\n  "additional_special_tokens": [\n    "<extra_id_0>",\n    "<extra_id_1>",\n    "<extra_id_2>",\n    "<extra_id_3>",\n    "<extra_id_4>",\n    "<extra_id_5>",\n    "<extra_id_6>",\n    "<extra_id_7>",\n    "<extra_id_8>",\n    "<extra_id_9>",\n    "<extra_id_10>",\n    "<extra_id_11>",\n    "<extra_id_12>",\n    "<extra_id_13>",\n    "<extra_id_14>",\n    "<extra_id_15>",\n    "<extra_id_16>",\n    "<extra_id_17>",\n    "<extra_id_18>",\n    "<extra_id_19>",\n    "<extra_id_20>",\n    "<extra_id_21>",\n    "<extra_id_22>",\n    "<extra_id_23>",\n    "<extra_id_24>",\n    "<extra_id_25>",\n    "<extra_

In [ ]:
try:
    s3.download_file(Bucket=bucket_name, Key=key)
    print('Zip file downloaded successfully.')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print('The specified object does not exist.')
    else:
        print('Error occurred:', e.response['Error']['Message'])
